# First-Entry Query

__Start Date:__ Jan 25, 2022

__Author:__ Julia Wang

__Goal:__ Converting the daily data dump of CDLI in [cdli_cat.csv](https://media.githubusercontent.com/media/cdli-gh/data/master/cdli_cat.csv) to quick statement in FactGrid.

In [1]:
import pandas as pd

## Read in the Data

In [2]:
url = 'https://media.githubusercontent.com/media/cdli-gh/data/master/cdli_cat.csv'

data = pd.read_csv(url, sep=',')
data.head(6)

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,14,16,17,19,22,23,24,25,26,27,28,29,31,32,33,37,39,42,45,46,49,52,53,54,55,57,58,59,60,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,...,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
0,NaN,NaN,NaN,NaN,21198/zz001q0dtm,"Englund, Robert K.",NaN,CDLI,"31x61x18; Lú A 14-16.30-32.48-50; M XVIII, auf...",NaN,...,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,18,no translation,61,NaN
1,NaN,NaN,NaN,NaN,21198/zz001q0dv4,"Englund, Robert K.",NaN,CDLI,30x48x13; Lú A 13-15.23-25.?; Fundstelle wie W...,NaN,...,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,13,no translation,48,NaN
2,NaN,NaN,NaN,NaN,21198/zz001q0dwn,"Englund, Robert K.",NaN,"Englund, Robert K. & Nissen, Hans J.","42x53x19; Vocabulary 9; Qa XVI,2, unter der Ab...",NaN,...,NaN,NaN,Archaic Vocabulary (witness),Text category: 15-09; Foreign ID: LVO 9,NaN,NaN,19,no translation,53,NaN
3,NaN,NaN,NaN,NaN,21198/zz001q0dx5,"Englund, Robert K.",NaN,CDLI,26x23x23; Lú A 9-10.?.?; Fundstelle wie W 9123...,NaN,...,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,23,no translation,23,NaN
4,NaN,NaN,NaN,NaN,21198/zz001q0dzp,"Englund, Robert K.",NaN,CDLI,"29x36x20; Lú A Vorläufer; Qa XVI,2, unter der ...",NaN,...,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,20,no translation,36,NaN
5,NaN,NaN,NaN,NaN,21198/zz001q0f0p,"Englund, Robert K.",NaN,CDLI,82x62x19; Lú A Vorläufer; Fundstelle wie W 912...,NaN,...,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,19,no translation,62,NaN


In [3]:
data = data.fillna('-')

The columns in the cdli data dump are as below:

In [4]:
data.columns

Index(['accession_no', 'accounting_period', 'acquisition_history',
       'alternative_years', 'ark_number', 'atf_source', 'atf_up', 'author',
       'author_remarks', 'cdli_collation', 'cdli_comments', 'citation',
       'collection', 'composite_id', 'condition_description', 'date_entered',
       'date_of_origin', 'date_remarks', 'date_updated', 'dates_referenced',
       'db_source', 'designation', 'dumb', 'dumb2', 'electronic_publication',
       'elevation', 'excavation_no', 'external_id', 'findspot_remarks',
       'findspot_square', 'genre', 'google_earth_collection',
       'google_earth_provenience', 'height', 'id', 'id_text2', 'id_text',
       'join_information', 'language', 'lineart_up', 'material', 'museum_no',
       'object_preservation', 'object_type', 'period', 'period_remarks',
       'photo_up', 'primary_publication', 'provenience', 'provenience_remarks',
       'publication_date', 'publication_history', 'published_collation',
       'seal_id', 'seal_information', 's

## Select Desired Columns from the Data Dump

Create a `desired_df` made up with columns we want, following the [CDLI_wikidata_format](https://docs.google.com/spreadsheets/d/107ly4G5j3im6Hbifqw1HaB66zuqzf7ijN6q8A-WvH8s/edit#gid=1318719558) described in the LOD Tablet Dictionary (FG Cuneiform).

In [5]:
desired_df = pd.DataFrame({'id_text': [], 'external_id': [], 'genre': [], 'subgenre': [], 'subgenre_remarks': [], 
                           'language': [], 'material': [], 'object_type': [], 'period': [], 
                           'atf_source': [], 'author': [], 'author_remarks': [], 'publication_history': [], 
                           'primary_publication': [], 'designation': [], 'publication_date': [], 'provenience': [], 
                           'collection': [], 'museum_no': [], 'accession_no': [], 'height': [], 'thickness': [], 'width': [],
                           'object_remarks': [], 'date_of_origin': [], 'translation_source': [], 'date_entered': [],
                           'dates_referenced': [], 'seal_id': [], 'accounting_period': [], 'citation': []
                            })

desired_columns = desired_df.columns
not_covered_columns = []

for col in desired_columns:
    # if a desired column is found in data, directly use that column in data
    if col in data.columns:
        desired_df[col] = data[col]
    # if a desired column is not found in data, record the column name
    else:
        not_covered_columns += [col]


desired_df = desired_df.fillna('')
desired_df


,id_text,external_id,genre,subgenre,subgenre_remarks,language,material,object_type,period,atf_source,...,thickness,width,object_remarks,date_of_origin,translation_source,date_entered,dates_referenced,seal_id,accounting_period,citation
0,1,-,Lexical,Archaic Lu2 A (witness),-,undetermined,clay,tablet,Uruk III (ca. 3200-3000 BC),"Englund, Robert K.",...,18,61,-,00.00.00.00,no translation,12/4/2001,00.00.00.00,-,-,-
1,2,-,Lexical,Archaic Lu2 A (witness),-,undetermined,clay,tablet,Uruk III (ca. 3200-3000 BC),"Englund, Robert K.",...,13,48,-,00.00.00.00,no translation,12/4/2001,00.00.00.00,-,-,-
2,3,-,Lexical,Archaic Vocabulary (witness),Text category: 15-09; Foreign ID: LVO 9,undetermined,clay,tablet,Uruk IV (ca. 3350-3200 BC),"Englund, Robert K.",...,19,53,-,-,no translation,12/4/2001,-,-,-,-
3,4,-,Lexical,Archaic Lu2 A (witness),-,undetermined,clay,tablet,Uruk IV (ca. 3350-3200 BC),"Englund, Robert K.",...,23,23,-,00.00.00.00,no translation,12/4/2001,00.00.00.00,-,-,-
4,5,-,Lexical,Archaic Lu2 A (witness),-,undetermined,clay,tablet,Uruk IV (ca. 3350-3200 BC),"Englund, Robert K.",...,20,36,-,00.00.00.00,no translation,12/4/2001,00.00.00.00,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353278,532443,-,Legal,-,-,Sumerian,clay,tablet,Old Babylonian (ca. 1900-1600 BC),no atf,...,-,-,-,-,no translation,8/13/2022,-,-,-,-
353279,532444,-,Administrative,-,-,Akkadian,clay,tablet,Old Babylonian (ca. 1900-1600 BC),no atf,...,20,34,-,-,no translation,8/13/2022,-,-,-,-
353280,532445,-,Administrative,-,-,Akkadian,clay,tablet,Old Babylonian (ca. 1900-1600 BC),no atf,...,20,34,-,-,no translation,8/13/2022,-,-,-,-
353281,532446,-,Legal,-,-,Akkadian,clay,tablet & envelope,Old Babylonian (ca. 1900-1600 BC),no atf,...,-,-,-,-,no translation,8/19/2022,-,-,-,-


We can see that all desired columns except for the ones below are available in `data` (cdli daily data dump).

In [6]:
not_covered_columns

[]

Next, we need to convert the content of the desired columns into FG formats. An example of FG can be found [here](https://database.factgrid.de/wiki/Item:Q471142). The rules for conversion is documented in the [demo_FGformat](https://docs.google.com/spreadsheets/d/107ly4G5j3im6Hbifqw1HaB66zuqzf7ijN6q8A-WvH8s/edit#gid=349399460) in the LOD Tablet Dictionary.


## Clean Up the Selected Columns


## 1. __language__ 

Reference the [Lang](https://docs.google.com/spreadsheets/d/107ly4G5j3im6Hbifqw1HaB66zuqzf7ijN6q8A-WvH8s/edit#gid=708139167) sheet in the LOD google sheet and make sure that all five columns are there:

* Parent language
* Dialect 
* Wikidata_link: use the link of the parent language and the entire link would be the QID
* Protocol_Code
* Inline_Code
* Lang_P155: (are we sure that this is indeed this language) if there is a question mark after the language, populate this column with the wikilink with Q22757.

For entries with '\x0b', can just remove these tags.

For entries that are "undetermined" or "unclear", there is no link (QID).

For entries with "pseudo", the corresponding tablets could be fake or indicating that the language is a pseudo language (e.g., pseudo Greek could be a person who does not really speak Greek writing in Greek). Like "undetermined" and "unclear", there is no link for this (i.e., we do not make a factual statement about this).

In [7]:
desired_df['language'].unique()

array(['undetermined', 'Sumerian', 'Sumerian ?', 'Akkadian', '-',
       'Sumerian; Akkadian', 'Elamite', 'Akkadian; Elamite', 'Hurrian',
       'Akkadian ?', 'Eblaite', 'Akkadian; Elamite; Persian; Egyptian ?',
       '\x0b', 'Ugaritic', 'undetermined (pseudo)', 'Aramaic',
       'Sumerian; Akkadian (pseudo)', 'no linguistic content',
       'Akkadian; Persian; Elamite; Egyptian ', 'Hebrew', 'Aramaic ?',
       'Akkadian; Aramaic', 'Hebrew ?', 'Greek', 'Phoenician', 'Sabaean',
       'Hittite', 'Akkadian; Egyptian', 'Hurrian ?', 'Qatabanian',
       'uncertain', 'Persian', 'Hittite ?', 'Hittite; Hattic', 'Syriac',
       'Mandaic', 'Hittite; Hurrian', 'uninscribed', 'Akkadian\x0b',
       'Akkadian\x0b\x0b', 'Sumerian; Akkadian ?', 'Egyptian ?',
       'Akkadian; Elamite; Persian', 'Arabic', 'Akkadian; Greek',
       'Urartian', 'Akkadian; Elamite; Persian; Egyptian',
       'Akkadian; Persian', 'Egyptian', 'Luwian',
       'Persian; Elamite; Akkadian', 'Akkadian; Persian; Elamite',
 

2. __material__

material = Material 

when there are multiples, separated by `:` 

the entry needs to be duplicated with both Q-ids for this field)

In [ ]:
desired_df['material'].unique()

array(['clay', 'gypsum', 'stone: steatite', 'stone: limestone', 'stone',
       'stone: onyx', 'stone: slate', 'stone: basalt', 'stone: alabaster',
       'clay\x0b', 'stone: agate', 'metal: bronze', 'stone: granite',
       'stone: agate ?', 'stone: diorite', 'stone: alabaster ?', '-',
       'metal', 'stone: calcite', 'stone: marble', 'stone: rock crystal',
       'stone: calcite ?', 'stone ?', 'bitumen', 'metal: copper',
       'stone: dolomite', 'stone: haematite', 'bone: shell',
       'stone: dolerite', 'metal: gold', 'stone: quartz',
       'stone: carnelian', 'metal: bronze ?', 'clay: terracotta',
       'stone: lapis lazuli', 'stone: serpentine', 'stone: hematite',
       'stone: jasper', 'stone: greenstone', 'stone: schist',
       'stone: crystal', 'stone: aragonite', 'stone: diorite ?',
       'stone: gypsum', 'stone: shale', 'stone: limestone ?',
       'stone: syenite', 'stone: gypsum ?', 'metal: silver',
       'stone: hornfels', 'stone: porphyry', 'stone: lava',
       

3. __collection__

collection = Museum (only wikidata QIDs needed)

In [ ]:
desired_df['collection'].unique()

array(['Vorderasiatisches Museum, Berlin, Germany',
       'National Museum of Iraq, Baghdad, Iraq',
       'National Museum of Iraq, Baghdad, Iraq; Vorderasiatisches Museum, Berlin, Germany',
       ..., 'Mardin Museum, Mardin, Turkey', 'Oylum Höyük, Oylum, Turkey',
       'private: William T. Grant Jr., Pelham Manor, New York, USA'],
      dtype=object)

In [ ]:
desired_df['provenience'].unique()

array(['Uruk (mod. Warka)', 'uncertain (mod. Jemdet Nasr)',
       'Ur (mod. Tell Muqayyar)', 'Kish (mod. Tell Uhaimir)',
       'Uruk (mod. Warka) ?', 'uncertain (mod. Tell Uqair) ?',
       'uncertain (mod. uncertain)', 'uncertain (mod. Jemdet Nasr) ?',
       'uncertain (mod. Tell Uqair)', 'Larsa (mod. Tell as-Senkereh)',
       'Larsa (mod. Tell as-Senkereh) ?', 'Ešnunna (mod. Tell Asmar)',
       'Umma (mod. Tell Jokha) ?', 'Nippur (mod. Nuffar)',
       'Ur (mod. Tell Muqayyar) ?', 'Tutub (mod. Khafaje)',
       'Šuruppak (mod. Fara) ?', 'Susa (mod. Shush)',
       'uncertain (mod. Chogha Mish)', 'Susa (mod. Shush) ?',
       'uncertain (mod. Hissar)', 'uncertain (mod. Godin Tepe)',
       'Anšan (mod. Tell Malyan)', '-', 'uncertain (mod. Shahr-i Sokhta)',
       'uncertain (mod. Tepe Sialk)', 'uncertain (mod. Tepe Yahya)',
       'Šuruppak (mod. Fara)', 'Kisurra (mod. Abu Hatab) ?',
       'uncertain (mod. Abu Salabikh)', 'Adab (mod. Bismaya) ?',
       'Girsu (mod. Tello)', 'Ad

In [ ]:
# provenience = CIGS_FG (match on neon green, and join light green columns; for origin see Geography notebook) 
# period = Period (match on neon green, and join light green columns; see Chronology notebook)
# date_of_origin = Rulers (may need updating… see